In [1]:
# July 2023
# working with LAS datasets (lidar datasets)

In [2]:
# source: https://medium.com/spatial-data-science/an-easy-way-to-work-and-visualize-lidar-data-in-python-eed0e028996c

In [3]:
# this laspy package that I'm using can work with .las data. But it does not support .laz data.

In [1]:
import laspy
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
file_addr = '/home/sobhan/Downloads/lidar data/AK_20140715_l6s82.las'
# file_addr = '/home/sobhan/Downloads/lidar data/InterpineData/LIR9_UNC_0_0.las'
# file_addr = '/home/sobhan/Downloads/lidar data/InterpineData/LIR9_UNC_0_0.las'

In [3]:
laspy.read(file_addr)

<LasData(1.4, point fmt: <PointFormat(1, 32 bytes of extra dims)>, 82556393 points, 1 vlrs)>

In [4]:
las = laspy.read(file_addr)

In [5]:
las.header

<LasHeader(1.4, <PointFormat(1, 32 bytes of extra dims)>)>

In [6]:
las.header.point_format

<PointFormat(1, 32 bytes of extra dims)>

In [7]:
las.header.point_count

82556393

In [8]:
las.vlrs

[<ExtraBytesVlr(extra bytes structs: 2)>]

In [9]:
# since the code format is 1.1 it is categorized as sth that must be read from
# https://pro.arcgis.com/en/pro-app/3.0/help/data/las-dataset/storing-lidar-data.htm

In [10]:
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'Range',
 'Ring',
 'ExtraBytes']

In [11]:
las.X

array([-9308, -8813, -8629, ...,  1607,  1642,  1681], dtype=int32)

In [12]:
las.Y,
las.Z

array([-111,  159, -222, ..., -126, -128, -116], dtype=int32)

In [13]:
# 1 = Unassigned
# 2 = Ground
# 5 = High Vegetation
# 6 = Building

las.classification

<SubFieldView([0 0 0 ... 0 0 0])>

In [14]:
# To create 3D point cloud data, we can stack together with the X, Y, and Z dimensions, 
# using Numpy like this

In [ ]:
point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
point_data

In [19]:
# as mentioned at https://pro.arcgis.com/en/pro-app/3.0/help/data/las-dataset/storing-lidar-data.htm
# the building class is ==6
buildings = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
buildings.points = las.points[las.classification == 6]

In [20]:
# now we have a new filtered las data which only contains buildings' data:
type(buildings)

laspy.lasdata.LasData

In [21]:
# so it has no building in it: as
buildings.header.point_count

0

In [22]:
# ground==2:
ground = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
ground.points = las.points[las.classification == 2]
ground.header.point_count

2

# visualization

In [23]:

point_data = np.stack([ground.X, ground.Y, ground.Z], axis=0).transpose((1, 0))


geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])

In [24]:
# all possible classfication in the given file:
ground = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
for i in range(0,254):
    ground.points = las.points[las.classification == i]
    n_points = ground.header.point_count
    if n_points>0:
        print(f'classifaction= {i}, {n_points}')

classifaction= 0, 4345578
classifaction= 1, 5464665
classifaction= 2, 2


In [25]:
# upspec==2:
upspec = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
upspec.points = las.points[las.classification == 2]
upspec.header.point_count

point_data = np.stack([upspec.X, upspec.Y, upspec.Z], axis=0).transpose((1, 0))


geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])

In [26]:
#  Ex2.

In [27]:
file_addr = '/home/sobhan/Downloads/lidar data/ME_20170816_Schoodic_c0r6.las'

In [28]:
las = laspy.read(file_addr)

In [29]:
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time',
 'ExtraBytes']

In [30]:
# all possible classfication in the given file:
new_las = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
for i in range(0,254):
    new_las.points = las.points[las.classification == i]
    n_points = new_las.header.point_count
    if n_points>0:
        print(f'classifaction= {i}, {n_points}')

classifaction= 1, 11873628
classifaction= 2, 3506729
